In [ ]:
import pandas as pd
import os
import joblib # Biblioteca para salvar o modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
# --- ADICIONAR ESTAS IMPORTACOES ---
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

# --- Passo 1: Carregar os Dados Limpos ---
caminho_dados_limpos = os.path.join('..', 'data', 'dados_limpos.csv')

try:
    df = pd.read_csv(caminho_dados_limpos)
    print("Arquivo de dados limpos carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{caminho_dados_limpos}'.")

# --- Passo 2: Preparação dos Dados para o Modelo ---

# Nosso objetivo (alvo) é prever 'risco_evasao_declarado'.
# Vamos simplificar para uma classificação binária: 'Sim' vs 'Não'.
# Por enquanto, vamos remover os 'Talvez' para ter um alvo claro.
df_modelo = df[df['risco_evasao_declarado'].isin(['Sim', 'Não'])].copy()

# Selecionar as 'features' (características) que usaremos para treinar o modelo.
# Excluímos colunas de texto livre, IDs ou a resposta original.
features = [
    'curso', 'periodo', 'idade', 'genero', 'cor_raca',
    'renda_familiar', 'trabalha'
]

# A variável que queremos prever (alvo ou target)
target = 'risco_evasao_declarado'

# Separar os dados em X (features) e y (target)
X = df_modelo[features]
y = df_modelo[target]

# Converter todas as colunas categóricas em números usando One-Hot Encoding.
# Isso cria novas colunas para cada categoria (ex: curso_Técnico_Informática, curso_Licenciatura...)
X_encoded = pd.get_dummies(X, drop_first=True)

# Guardar os nomes das colunas usadas no modelo, para usarmos no dashboard depois
model_columns = X_encoded.columns
joblib.dump(model_columns, os.path.join('..', 'dashboard', 'colunas_modelo.joblib'))

# --- NOVO: APLICAR SMOTE PARA BALANCEAR OS DADOS ---
print("\nAplicando SMOTE para balancear os dados...")
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_encoded, y)

print(f"Distribuição original: {np.bincount([1 if val == 'Sim' else 0 for val in y])}")
print(f"Distribuição após SMOTE: {np.bincount([1 if val == 'Sim' else 0 for val in y_balanced])}")

# --- Passo 3: Dividir os Dados em Treino e Teste ---
# 80% dos dados para treinar o modelo, 20% para testar sua performance.
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

print(f"\nTamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

# --- NOVO: VALIDAÇÃO CRUZADA ---
print("\nRealizando validação cruzada...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model_cv = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42
)
cv_scores = cross_val_score(model_cv, X_balanced, y_balanced, cv=cv, scoring='f1_macro')
print(f"F1-Score médio na validação cruzada: {cv_scores.mean():.2f} (+/- {cv_scores.std() * 2:.2f})")

# --- Passo 4: Treinar o Modelo de Machine Learning ---
print("\nIniciando o treinamento do modelo RandomForestClassifier...")

# SUBSTITUIR pelo modelo com hiperparâmetros ajustados
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42
)

# Treina o modelo com os dados de treino
model.fit(X_train, y_train)

print("Treinamento concluído!")

# --- Passo 5: Avaliar a Performance do Modelo ---
print("\nAvaliando o modelo com os dados de teste...")

# Faz as previsões nos dados de teste (dados que o modelo nunca viu)
y_pred = model.predict(X_test)

# Compara as previsões com os resultados reais
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo: {accuracy:.2f}")

# NOVO: Adicionar matriz de confusão
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# --- Passo 6: Salvar o Modelo Treinado ---
# Salva o objeto do modelo treinado em um arquivo para ser usado pelo dashboard.
caminho_modelo = os.path.join('..', 'dashboard', 'modelo_evasao.joblib')
joblib.dump(model, caminho_modelo)

print(f"\nModelo salvo com sucesso em: '{caminho_modelo}'")

Arquivo de dados limpos carregado com sucesso!

Tamanho do conjunto de treino: 45 amostras
Tamanho do conjunto de teste: 12 amostras

Iniciando o treinamento do modelo RandomForestClassifier...
Treinamento concluído!

Avaliando o modelo com os dados de teste...

Acurácia do modelo: 0.50

Relatório de Classificação:
              precision    recall  f1-score   support

         Não       0.60      0.75      0.67         8
         Sim       0.00      0.00      0.00         4

    accuracy                           0.50        12
   macro avg       0.30      0.38      0.33        12
weighted avg       0.40      0.50      0.44        12


Modelo salvo com sucesso em: '..\dashboard\modelo_evasao.joblib'
